# EJERCICIO 5 - GUILLERMO ARREDONDO VAZQUEZ
### Dada la siguiente lista, crea un RDD y realiza lo siguiente:
#### Inicializa una sesión de spark en consola y coloca en una variable la información de cada archivo csv que se encuentran en el siguiente link
- Realiza una unión para unir todos los pokemon de cada generación
- Devuelve un conteo por generación y muestra el resultado en pantalla
- Devuelve un conteo por Type_1 y muestra el resultado en pantalla
- Devuelve un conteo por pokemon legendarios y muestra el resultado en pantalla
- Devuelve el pokemon con más puntos de:
- Vida (hp), Ataque (attack), Defensa (defense)
- Devuelve el pokemon con menos puntos de:
- Ataque especial (sp_attack), Defensa especial (sp_defense), Velocidad (speed)
- Realiza un Join de la información de los pokemon y sus ventajas
- Del resultado final, devuelve toda la información del pokemon, pero solo la información de las primeras 2 ventajas, las primeras 2 desventajas y el campo “sin_efecto”
- Escribe el resultado en un archivo parquet pkmn_result__info.parquet

In [1]:
import findspark
findspark.init()
import pandas as pd
import pyspark

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
#Libreria para utilizar el count
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.getOrCreate()

In [16]:
#dfInfo =spark.read.csv("data/t_players_info.csv", sep=",", header=True, inferSchema=True)
pk1 = spark.read.csv("data/pkmn_gen_1.csv", sep=",",inferSchema=True,header=True)
pk2 = spark.read.csv("data/pkmn_gen_2.csv", sep=",",inferSchema=True,header=True)
pk3 = spark.read.csv("data/pkmn_gen_3.csv", sep=",",inferSchema=True,header=True)
pk4 = spark.read.csv("data/pkmn_gen_4.csv", sep=",",inferSchema=True,header=True)
pk5 = spark.read.csv("data/pkmn_gen_5.csv", sep=",",inferSchema=True,header=True)
pk6 = spark.read.csv("data/pkmn_gen_6.csv", sep=",",inferSchema=True,header=True)
ventajas = spark.read.csv("data/ventajas.csv", sep=",",inferSchema=True,header=True)

In [17]:
#Realiza una unión para unir todos los pokemon de cada generación
#pk2.show()
allpk = pk1.union(pk2).union(pk3).union(pk4).union(pk5).union(pk6)
allpk.sort(desc("id")).show()

+---+--------------------+-------+------+------+------+-------+------+------+------+---------+
| id|                name| type_1|type_2|    hp|attack|defense|sp_atk|sp_def| speed|legendary|
+---+--------------------+-------+------+------+------+-------+------+------+------+---------+
|721|           Volcanion|   Fire| Water| 80,00|110,00| 120,00|130,00| 90,00| 70,00|     true|
|720|  HoopaHoopa Unbound|Psychic|  Dark| 80,00|160,00|  60,00|170,00|130,00| 80,00|     true|
|720| HoopaHoopa Confined|Psychic| Ghost| 80,00|110,00|  60,00|150,00|130,00| 70,00|     true|
|719|             Diancie|   Rock| Fairy| 50,00|100,00| 150,00|100,00|150,00| 50,00|     true|
|719| DiancieMega Diancie|   Rock| Fairy| 50,00|160,00| 110,00|160,00|110,00|110,00|     true|
|718|    Zygarde50% Forme| Dragon|Ground|108,00|100,00| 121,00| 81,00| 95,00| 95,00|     true|
|717|             Yveltal|   Dark|Flying|126,00|131,00|  95,00|131,00| 98,00| 99,00|     true|
|716|             Xerneas|  Fairy|  null|126,00|13

In [5]:
#Devuelve un conteo por generación y muestra el resultado en pantalla
gener = [pk1,pk2,pk3,pk4,pk5,pk6]
contador = 1
for i in gener:
    val = i
    print("La generación",contador,"tiene",val.count(),"pokemones")
    contador = contador+1
print("En total son",allpk.count(),"pokemones")

La generación 1 tiene 166 pokemones
La generación 2 tiene 106 pokemones
La generación 3 tiene 160 pokemones
La generación 4 tiene 121 pokemones
La generación 5 tiene 165 pokemones
La generación 6 tiene 82 pokemones
En total son 800 pokemones


In [6]:
#Devuelve un conteo por Type_1 y muestra el resultado en pantalla
allpk.groupBy("Type_1").agg(count("*").alias("Number/type1")).show()

+--------+------------+
|  Type_1|Number/type1|
+--------+------------+
|   Water|         112|
|  Poison|          28|
|    Rock|          44|
|     Ice|          24|
|   Ghost|          32|
| Psychic|          57|
|  Dragon|          32|
|     Bug|          69|
|Electric|          44|
|    Fire|          52|
|  Ground|          32|
|Fighting|          27|
|   Grass|          70|
|  Normal|         100|
|   Steel|          27|
|   Fairy|          15|
|    Dark|          31|
|  Flying|           4|
+--------+------------+



In [7]:
#Devuelve un conteo por pokemon legendarios y muestra el resultado en pantalla
legendary = allpk.filter(allpk.legendary == True)
print("Cantidad de pokemon legendarios =",legendary.count())

Cantidad de pokemon legendarios = 65


In [8]:
#Devuelve el pokemon con más puntos de:
#Vida (hp), Ataque (attack), Defensa (defense)
#LO PRIMERO ES ACTUALIZAR LOS DATOS DE LA TABLA, QUITAR LAS COMAS Y CONVERTIR LOS DATOS A INT
new = allpk.withColumn("hp", regexp_replace(col("hp"), ",", "").cast("integer"))
new = new.withColumn("hp", col("hp") / 100)
new = new.withColumn("attack", regexp_replace(col("attack"), ",", "").cast("integer"))
new = new.withColumn("attack", col("attack") / 100)
new = new.withColumn("defense", regexp_replace(col("defense"), ",", "").cast("integer"))
new = new.withColumn("defense", col("defense") / 100)
#new.dtypes
strong = new.orderBy(new["hp"]+new["attack"]+new["defense"],ascending=False).first()
print("El pokemon con mas puntos es",strong["name"],"con",strong["hp"]+strong["attack"]+strong["defense"],"puntos")
#new.orderBy(new["hp"]+new["attack"]+new["defense"],ascending=False).show()


El pokemon con mas puntos es AggronMega Aggron con 440.0 puntos


In [9]:
#Devuelve el pokemon con menos puntos de:
#Ataque especial (sp_attack), Defensa especial (sp_defense), Velocidad (speed)
#LO PRIMERO ES ACTUALIZAR LOS DATOS DE LA TABLA, QUITAR LAS COMAS Y CONVERTIR LOS DATOS A INT
new = new.withColumn("sp_atk", regexp_replace(col("sp_atk"), ",", "").cast("integer"))
new = new.withColumn("sp_atk", col("sp_atk") / 100)
new = new.withColumn("sp_def", regexp_replace(col("sp_def"), ",", "").cast("integer"))
new = new.withColumn("sp_def", col("sp_def") / 100)
new = new.withColumn("speed", regexp_replace(col("speed"), ",", "").cast("integer"))
new = new.withColumn("speed", col("speed") / 100)
#new.dtypes
weakest = new.orderBy(new["sp_atk"]+new["sp_def"]+new["speed"], ascending=True).first()
print("El pokemon con menos puntos es",weakest["name"])
#new.orderBy(new["sp_atk"]+new["sp_def"]+new["speed"], ascending=True).show()

El pokemon con menos puntos es Silcoon


In [26]:
#Realiza un Join de la información de los pokemon y sus ventajas
allpkNew = allpk.join(ventajas, allpk["type_1"] == ventajas["tipo"])
allpkNew.show(5)

+---+--------------------+------+------+-----+------+-------+------+------+------+---------+----+---------+---------+---------+---------+---------+------------+------------+------------+------------+------------+----------+
| id|                name|type_1|type_2|   hp|attack|defense|sp_atk|sp_def| speed|legendary|tipo|ventaja_1|ventaja_2|ventaja_3|ventaja_4|ventaja_5|desventaja_1|desventaja_2|desventaja_3|desventaja_4|desventaja_5|sin_efecto|
+---+--------------------+------+------+-----+------+-------+------+------+------+---------+----+---------+---------+---------+---------+---------+------------+------------+------------+------------+------------+----------+
|  4|          Charmander|  Fire|  null|39,00| 52,00|  43,00| 60,00| 50,00| 65,00|    false|Fire|   Planta|    Hielo|    Bicho|    Acero|  Ninguna|       Fuego|        Agua|        Roca|      Dragón|        null|   Ninguno|
|  5|          Charmeleon|  Fire|  null|58,00| 64,00|  58,00| 80,00| 65,00| 80,00|    false|Fire|   Plan

In [27]:
#Del resultado final, devuelve toda la información del pokemon,
#pero solo la información de las primeras 2 ventajas, las primeras 2 desventajas y el campo “sin_efecto”
allpkS = allpkNew.select("id","name","type_1","type_2","hp","attack","defense",
                         "sp_atk","sp_def","speed","legendary","ventaja_1","ventaja_2","desventaja_1","desventaja_2")
allpkS.show(5)

+---+--------------------+------+------+-----+------+-------+------+------+------+---------+---------+---------+------------+------------+
| id|                name|type_1|type_2|   hp|attack|defense|sp_atk|sp_def| speed|legendary|ventaja_1|ventaja_2|desventaja_1|desventaja_2|
+---+--------------------+------+------+-----+------+-------+------+------+------+---------+---------+---------+------------+------------+
|  4|          Charmander|  Fire|  null|39,00| 52,00|  43,00| 60,00| 50,00| 65,00|    false|   Planta|    Hielo|       Fuego|        Agua|
|  5|          Charmeleon|  Fire|  null|58,00| 64,00|  58,00| 80,00| 65,00| 80,00|    false|   Planta|    Hielo|       Fuego|        Agua|
|  6|           Charizard|  Fire|Flying|78,00| 84,00|  78,00|109,00| 85,00|100,00|    false|   Planta|    Hielo|       Fuego|        Agua|
|  6|CharizardMega Cha...|  Fire|Dragon|78,00|130,00| 111,00|130,00| 85,00|100,00|    false|   Planta|    Hielo|       Fuego|        Agua|
|  6|CharizardMega Cha...| 

In [25]:
#Escribe el resultado en un archivo parquet pkmn_result__info.parquet
allpkS.write.parquet("C:/Users/SoporteTI/Entrega/Examen_Spark/pkmn_result__info")